In [1]:
!pip install nltk
!pip install scikit-learn
!pip install imbalanced-learn

In [2]:
import pandas as pd
import numpy as np
import nltk
import re
import string
from sklearn.model_selection import train_test_split

In [3]:
dataset = pd.read_csv("sentiment_analysis.csv")

### Preprocessing

In [4]:
dataset.duplicated().sum()

0

In [5]:
dataset.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [6]:
# Convert all columns to lowercase
dataset = dataset.applymap(lambda x: x.lower() if isinstance(x, str) else x)


C:\Users\pathu\AppData\Local\Temp\ipykernel_12608\1336752758.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dataset = dataset.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [7]:
dataset.head(10)

,id,label,tweet
0,1,0,#fingerprint #pregnancy test https://goo.gl/h1...
1,2,0,finally a transparant silicon case ^^ thanks t...
2,3,0,we love this! would you go? #talk #makememorie...
3,4,0,i'm wired i know i'm george i was made that wa...
4,5,1,what amazing service! apple won't even talk to...
5,6,1,iphone software update fucked up my phone big ...
6,7,0,happy for us .. #instapic #instadaily #us #son...
7,8,0,new type c charger cable #uk http://www.ebay.c...
8,9,0,bout to go shopping again listening to music #...
9,10,0,photo: #fun #selfie #pool #water #sony #camera...


In [8]:
dataset['tweet'] = dataset['tweet'].replace(r'http\S+', '', regex=True)

In [9]:
dataset.head(10)

,id,label,tweet
0,1,0,#fingerprint #pregnancy test #android #apps #...
1,2,0,finally a transparant silicon case ^^ thanks t...
2,3,0,we love this! would you go? #talk #makememorie...
3,4,0,i'm wired i know i'm george i was made that wa...
4,5,1,what amazing service! apple won't even talk to...
5,6,1,iphone software update fucked up my phone big ...
6,7,0,happy for us .. #instapic #instadaily #us #son...
7,8,0,new type c charger cable #uk … #bay #amazon #...
8,9,0,bout to go shopping again listening to music #...
9,10,0,photo: #fun #selfie #pool #water #sony #camera...


In [10]:
def remove_punctuation(text):
  """Removes punctuation characters from a string.

  Args:
      text: The string to remove punctuation from.

  Returns:
      The string with punctuation removed.
  """
  # Define punctuation characters to remove
  punct = string.punctuation
  # Use translate method with a mapping table to remove punctuation
  return text.translate(str.maketrans('', '', punct))


In [11]:
# Assuming your data has a column named 'tweet'
dataset['tweet'] = dataset['tweet'].apply(remove_punctuation)


In [12]:
dataset.head(10)

,id,label,tweet
0,1,0,fingerprint pregnancy test android apps beaut...
1,2,0,finally a transparant silicon case thanks to ...
2,3,0,we love this would you go talk makememories un...
3,4,0,im wired i know im george i was made that way ...
4,5,1,what amazing service apple wont even talk to m...
5,6,1,iphone software update fucked up my phone big ...
6,7,0,happy for us instapic instadaily us sony xper...
7,8,0,new type c charger cable uk … bay amazon etsy...
8,9,0,bout to go shopping again listening to music i...
9,10,0,photo fun selfie pool water sony camera picoft...


In [13]:
dataset['tweet'] = dataset['tweet'].str.replace('\d+', ' ', regex = True)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\pathu\AppData\Local\Temp\ipykernel_12608\3263135453.py:1: SyntaxWarning: invalid escape sequence '\d'
  dataset['tweet'] = dataset['tweet'].str.replace('\d+', ' ', regex = True)


In [14]:
dataset.head(10)

,id,label,tweet
0,1,0,fingerprint pregnancy test android apps beaut...
1,2,0,finally a transparant silicon case thanks to ...
2,3,0,we love this would you go talk makememories un...
3,4,0,im wired i know im george i was made that way ...
4,5,1,what amazing service apple wont even talk to m...
5,6,1,iphone software update fucked up my phone big ...
6,7,0,happy for us instapic instadaily us sony xper...
7,8,0,new type c charger cable uk … bay amazon etsy...
8,9,0,bout to go shopping again listening to music i...
9,10,0,photo fun selfie pool water sony camera picoft...


In [15]:
dataset["tweet"]

0       fingerprint pregnancy test  android apps beaut...
1       finally a transparant silicon case  thanks to ...
2       we love this would you go talk makememories un...
3       im wired i know im george i was made that way ...
4       what amazing service apple wont even talk to m...
                              ...                        
7915    live out loud lol liveoutloud selfie smile son...
7916    we would like to wish you an amazing day make ...
7917    helping my lovely   year old neighbor with her...
7918    finally got my smart pocket wifi stay connecte...
7919    apple barcelona apple store bcn barcelona trav...
Name: tweet, Length: 7920, dtype: object

In [16]:
nltk.download('stopwords', download_dir = '../static/model')

[nltk_data] Downloading package stopwords to ../static/model...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
dataset["tweet"]

0       fingerprint pregnancy test  android apps beaut...
1       finally a transparant silicon case  thanks to ...
2       we love this would you go talk makememories un...
3       im wired i know im george i was made that way ...
4       what amazing service apple wont even talk to m...
                              ...                        
7915    live out loud lol liveoutloud selfie smile son...
7916    we would like to wish you an amazing day make ...
7917    helping my lovely   year old neighbor with her...
7918    finally got my smart pocket wifi stay connecte...
7919    apple barcelona apple store bcn barcelona trav...
Name: tweet, Length: 7920, dtype: object

In [18]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [19]:
dataset["tweet"] = dataset["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw)) 

In [20]:
dataset["tweet"]

0       fingerprint pregnancy test android apps beauti...
1       finally transparant silicon case thanks uncle ...
2       love would go talk makememories unplug relax i...
3       im wired know im george made way iphone cute d...
4       amazing service apple wont even talk question ...
                              ...                        
7915    live loud lol liveoutloud selfie smile sony mu...
7916    would like wish amazing day make every minute ...
7917    helping lovely year old neighbor ipad morning ...
7918    finally got smart pocket wifi stay connected a...
7919    apple barcelona apple store bcn barcelona trav...
Name: tweet, Length: 7920, dtype: object

In [21]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [22]:
dataset["tweet"] = dataset["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))

In [23]:
dataset

,id,label,tweet
0,1,0,fingerprint pregnanc test android app beauti c...
1,2,0,final transpar silicon case thank uncl yay son...
2,3,0,love would go talk makememori unplug relax iph...
3,4,0,im wire know im georg made way iphon cute dave...
4,5,1,amaz servic appl wont even talk question unles...
...,...,...,...
7915,7916,0,live loud lol liveoutloud selfi smile soni mus...
7916,7917,0,would like wish amaz day make everi minut coun...
7917,7918,0,help love year old neighbor ipad morn made rea...
7918,7919,0,final got smart pocket wifi stay connect anyti...


### buld vocabulary

In [24]:
from collections import Counter
vocab = Counter()

In [25]:
for sentence in dataset["tweet"]:
    vocab.update(sentence.split())

In [26]:
tokens = [key for key in vocab if vocab[key] > 10] 

In [27]:
def save_vocabulary(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w', encoding="utf-8")
    file.write(data)
    file.close()

save_vocabulary(tokens, '../static/model/vocabulary.txt')

### deviding dataset

In [28]:
X = dataset['tweet']
Y = dataset['label']

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train , Y_test = train_test_split(X, Y, test_size = 0.2)

In [30]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []

    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1

        vectorized_lst.append(sentence_lst)

    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    return vectorized_lst
    

In [31]:
vectorized_x_train = vectorizer(X_train, tokens)

In [32]:
Y_train

3638    0
7460    0
1271    1
316     1
1330    0
       ..
82      1
159     0
5804    0
290     0
5342    0
Name: label, Length: 6336, dtype: int64

In [33]:
vectorized_x_test = vectorizer(X_test, tokens)

In [34]:
Y_train.value_counts()

label
0    4732
1    1604
Name: count, dtype: int64

In [35]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
vectorized_x_train_smote, y_train_smote = smote.fit_resample(vectorized_x_train, Y_train)


In [ ]:
y_train_smote.value_count(